In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np

import xarray as xr

import regionmask

import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cartopy.feature as cfeat
import matplotlib.patches as mpatches
import datetime

import feather

import glob


import sys
sys.path.append('../')
import utils
import plotting

np.seterr(invalid='ignore'); # disable a warning from matplotlib and cartopy

# Concentration Data

In [2]:
ds_normal_model = utils.import_GC_runs_general('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/','GEOSChem.SpeciesConc.2016*_0000z.nc4','GEOSChem.AerosolMass.2016*_0000z.nc4','normal_model' )
ds_nonuc_model = utils.import_GC_runs_general('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuclear/OutputDir/','GEOSChem.SpeciesConc.2016*_0000z.nc4','GEOSChem.AerosolMass.2016*_0000z.nc4','nonuc_model' )

datasets = [
    ds_normal_model,
    ds_nonuc_model
]

In [91]:
poll_ds = utils.combine_and_convert_ds(utils.gas_species_list, utils.aerosol_species_dict.keys(),
                                       datasets,['normal_model', 'nonuc_model'], 
                                       ['nonuc_model'], 
                                       'normal_model', 
                                       'normal and nonuclear' )



In [93]:
poll_ds.to_zarr('./data/nonuc_normal_GC.zarr', mode = 'w') #save the dataset 

# HEMCO Data

In [2]:
ds_normal = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/HEMCO_diagnostics.2016*.nc', combine = 'by_coords')
ds_nonuc = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuclear/OutputDir/HEMCO_diagnostics.2016*.nc', combine = 'by_coords')

ds_normal = ds_normal.isel(lev = 0)
ds_nonuc = ds_nonuc.isel(lev = 0)

ds_emis = xr.concat([ds_nonuc, ds_normal], pd.Index(['nonuc', 'normal'], name='model_name'))

In [3]:
emis_list = ['EmisNO2_Anthro', 'EmisNO_Anthro', 'EmisSO2_Anthro',  'EmisNH3_Total']

In [10]:
emis_ds = ds_emis.drop_vars([species for species in ds_emis.data_vars if species not in emis_list])
emis_ds['EmisNOx_Anthro'] = (
        emis_ds['EmisNO2_Anthro'] + emis_ds['EmisNO_Anthro'] #sum our NO2 and NO to get NOx
                   )
emis_ds['EmisNOx_Anthro'].attrs['units'] = 'kg/m2/s'

In [11]:
emis_ds.to_zarr('./data/emissions_HEMCO', mode = 'w') #save the dataset 

# US-EGO Emissions Data

In [2]:
##### import files and change hydro #####

###get ORIS, plant type, and Region Name from our modified generation file
oris_nonuc_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_all-generators_20k-new_name.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_normal_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_normal.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_nonuc_df.loc[oris_nonuc_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_normal_df.loc[oris_normal_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label

###import egrid generation data and modify the index to be ORISCode
egrid_df_raw = pd.read_excel('../raw_data/egrid2016_data.xlsx',sheet_name='GEN16', usecols='D, J, L')
egrid = egrid_df_raw.drop(egrid_df_raw.index[0]).rename(columns={'DOE/EIA ORIS plant or facility code':'ORISCode'})
#change those with no capacity to 1 (so that our capacity factor calculation doesn't fail)
egrid.loc[egrid['Generator nameplate capacity (MW)'] == 0, 'Generator nameplate capacity (MW)'] = 1
#calculate the capacity factor of generation in egrid
egrid['egrid_capafactor'] = egrid['Generator annual net generation (MWh)'] / (8760 * egrid['Generator nameplate capacity (MW)']) 

###import the generation data from our optimization
gen_normal_df=feather.read_dataframe('../optimization_model/outputs/gen_normal.feather')

###import the generation data from our optimization
gen_nonuc_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear_modified-all-generators.feather')



In [3]:
##### change index and convert to xarray #####

#group by ORIS code and make a date and ORIS code mult index
gen_nonuc_df = pd.concat([gen_nonuc_df,oris_nonuc_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_normal_df = pd.concat([gen_normal_df,oris_normal_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_normal_df = gen_normal_df.stack()
gen_nonuc_df = gen_nonuc_df.stack()
gen_nonuc_df.index.names = (['ORISCode','date'])
gen_normal_df.index.names = (['ORISCode','date'])

egrid = egrid.groupby('ORISCode').sum().drop(columns = 'egrid_capafactor') #drop capacity factor because that isn't the sum
 
#convert to xarray datasets
oris_nonuc_ds = oris_nonuc_df.to_xarray()
oris_normal_ds = oris_normal_df.to_xarray()
egrid_ds = egrid.to_xarray()
gen_normal_ds = gen_normal_df.to_xarray()
gen_nonuc_ds = gen_nonuc_df.to_xarray()

In [61]:
def combine_egrid_generation(oris_ds, gen_ds, egrid_ds):   
    #create a capacity, fueltype, and regionname grouped by ORISCode
    capacity = oris_ds.groupby('ORISCode').sum()['Capacity']
    fueltype = oris_ds.to_dataframe().groupby('ORISCode').first()['FuelType']
    regionname = oris_ds.to_dataframe().groupby('ORISCode').first()['RegionName']
    #group by ORIS code and take the mean of everything but capacity
    oris_ds = oris_ds.groupby('ORISCode').mean().drop('Capacity')
    #rename our generation variable
    gen_ds = gen_ds.rename('modelgeneration')

    ###concatenate the generation and ORIS files
    gmodel_oris_ds = xr.merge([gen_ds, oris_ds])
    #add in the capacity
    gmodel_oris_ds['Capacity'] = capacity
    #create a column for capacity factors
    gmodel_oris_ds['model_capafactor'] = 100 * gmodel_oris_ds['modelgeneration'] / (gmodel_oris_ds['Capacity'] * 8760) # % generation for each year's total capacity
    
    ###concatenate our model/oris and egrid emissions dataframes into one, grouped by ORIS code
    gmodel_egrid_ds = xr.merge([gmodel_oris_ds, egrid_ds])
    #turn all zeroes (just in the modelgeneration) to NAN
    gmodel_egrid_ds.where('modelgeneration' == 0)['modelgeneration'] = np.nan
    #rename the egrid data column for ease
    gmodel_egrid_ds = gmodel_egrid_ds.rename({'Generator annual net generation (MWh)':'annual_egridgeneration'})
    #add in fueltype
    gmodel_egrid_ds['fueltype'] = fueltype
    gmodel_egrid_ds = gmodel_egrid_ds.set_coords('fueltype')
    #add in region name
    gmodel_egrid_ds['regionname'] = regionname
    gmodel_egrid_ds = gmodel_egrid_ds.set_coords('regionname')
    return(gmodel_egrid_ds)
    

In [64]:
#### combine ourr egrid and generation info for no nuclear and normal models ####
nonuc_gmodel_egrid_ds = combine_egrid_generation(oris_nonuc_ds, gen_nonuc_ds, egrid_ds)
normal_gmodel_egrid_ds = combine_egrid_generation(oris_normal_ds, gen_normal_ds, egrid_ds)
gmodel_egrid_raw_ds = xr.concat([nonuc_gmodel_egrid_ds, normal_gmodel_egrid_ds], pd.Index(['nonuc','normal'], name='model_name'))

/home/emfreese/anaconda3/envs/grid_model/lib/python3.8/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [65]:
#### fix datetime and add dec 31st at the 23rd hour ####

###create a dict for dec 31st 23rd hour (just copy the dec 31 22nd hour data over)
for name in models:
    dec23_ds = gmodel_egrid_raw_ds.copy().isel(date = [-1])
    #change date to datetime for the 23rd hour
    dec23_ds['date'] = [datetime.datetime(2017,12,31,23,0)]
    
###change all dates to datetime
#make a list of dates for the year
base = datetime.datetime(2017, 1, 1) #base date
date_list = [base + datetime.timedelta(hours=x) for x in range(8759)] #loop through all hours but the final one (we will add it in above)
date_list
for name in models:
    gmodel_egrid_raw_ds['date'] = date_list
    gmodel_egrid_ds = xr.merge([gmodel_egrid_raw_ds, dec23_ds])

Note:

To convert our generation in MWh to get the emissions in kg/sec, we use the following (and making sure our emissions rates are in kg/sec)

$MWH/3600sec -> (MW/s) * kg/MW -> kg/s$


In [66]:
###### add NOx, SO2, CO2, CH4 emissions to gmodel_egrid dataset ######

no_mult = 0.8544304 # NO/NOx as estimated from NEI2011 inventory
no2_mult = 1 - 0.8544304 # NO2/NOx as estimated from NEI2011 inventory

### process our emissions by multiplying generation* emissions factors
gmodel_egrid_ds['model_NO_rate']  = no_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_NO2_rate']  = no2_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_SO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['model_CO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['model_CH4_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

gmodel_egrid_ds['egrid_annual_NO_rate']  = no_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_NO2_rate']  = no2_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_SO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['egrid_annual_CO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['egrid_annual_CH4_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

##### annual generation in original dataset ######
gmodel_egrid_ds['annual_modelgeneration'] = gmodel_egrid_ds['modelgeneration'].groupby('ORISCode').sum(dim = 'date')

/home/emfreese/anaconda3/envs/grid_model/lib/python3.8/site-packages/xarray/core/common.py:671: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  return self._groupby_cls(


In [90]:
####### create new datasets for region and fuel type grouped data, add annual generation to normal ds ######

###list of pollutants
pollutants = ['NO','NO2','SO2','CO2','CH4']

def create_grouped_ds(grouper, model):
    grouped_ds = gmodel_egrid_ds.copy().sel(model_name = model).groupby(grouper).sum()
    grouped_ds = grouped_ds.drop(['model_capafactor','PLCH4RTA','PLCO2RTA','PLNOXRTA','PLSO2RTA','PLN2ORTA'])
    return(grouped_ds)

###create datasets of grouped fuel type and region name, combine
nonuc_fueltype_grouped_ds = create_grouped_ds('fueltype', 'nonuc')
normal_fueltype_grouped_ds = create_grouped_ds('fueltype', 'normal')

nonuc_regionname_grouped_ds = create_grouped_ds('regionname', 'nonuc')
normal_regionname_grouped_ds = create_grouped_ds('regionname', 'normal')

fueltype_grouped_ds = xr.concat([nonuc_fueltype_grouped_ds, normal_fueltype_grouped_ds], pd.Index(['nonuc','normal'], name='model_name'))
regionname_grouped_ds = xr.concat([nonuc_regionname_grouped_ds, normal_regionname_grouped_ds], pd.Index(['nonuc','normal'], name='model_name'))

In [113]:
#add annual emissions of each pollutant
for pollutant in pollutants:
    regionname_grouped_ds[f'model_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'model_{pollutant}_rate']*86400).sum(dim = 'date')
    fueltype_grouped_ds[f'model_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'model_{pollutant}_rate']*86400).sum(dim = 'date')
    gmodel_egrid_ds[f'model_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'model_{pollutant}_rate']*86400).sum(dim = 'date')

    regionname_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'egrid_annual_{pollutant}_rate']*86400*365)
    fueltype_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'egrid_annual_{pollutant}_rate']*86400*365)
    gmodel_egrid_ds[f'egrid_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'egrid_annual_{pollutant}_rate']*86400*365)

#add ratio of difference in emissions normal-nonuc/ difference in generation normal-nonuc to the nonuc dataset
for pollutant in pollutants:
    fueltype_grouped_ds.sel(model_name = 'nonuc')[f'normal-nonuc_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'nonuc')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'nonuc')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'nonuc')[f'normal-nonuc_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'nonuc')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'nonuc')['annual_modelgeneration'])



In [114]:
#### add attributes ####
for pollutant in pollutants:
    if pollutant == 'NO2' or 'NO':
                gmodel_egrid_ds[f'PLNOXRTA'].attrs['units'] = 'kg/Mwh'
    else:
        gmodel_egrid_ds[f'PL{pollutant}RTA'].attrs['units'] = 'kg/Mwh'
    for idx, ds in enumerate([regionname_grouped_ds, fueltype_grouped_ds, gmodel_egrid_ds]):
        ds[f'model_{pollutant}_rate'].attrs['units'] = 'kg/s'
        ds[f'modelgeneration'].attrs['units'] = 'Mwh'
        for typem in ['egrid','model']:
            ds[f'annual_{typem}generation'].attrs['units'] = 'Mwh'
            ds[f'{typem}_annual_{pollutant}_conc'].attrs['units'] = 'kg/yr'
        ds.attrs['model'] = name
        ds.attrs['group'] = ['Region','Fuel Type','All'][idx]

In [122]:
##### save datasets #####

### Final datasets to save are: gmodel_egrid, fueltype_grouped_ds, regionname_grouped_ds
regionname_grouped_ds.to_netcdf(f'./data/regionname_grouped_emissions_ds.nc', 'w')
fueltype_grouped_ds.to_netcdf(f'./data/fueltype_grouped_emissions_ds.nc', 'w')
gmodel_egrid_ds.to_netcdf(f'./data/gmodel_egrid_emissions_ds.nc', 'w')

### datasets partway to save are:

oris_nonuc_ds.to_netcdf('../optimization_model/outputs/oris_nonuc_ds.nc', 'w')
oris_normal_ds.to_netcdf('../optimization_model/outputs/oris_normal_ds.nc', 'w')
egrid_ds.to_netcdf('../optimization_model/outputs/egrid_ds.nc', 'w')
gen_normal_ds.to_netcdf('../optimization_model/outputs/gen_normal_ds.nc', 'w')
gen_nonuc_ds.to_netcdf('../optimization_model/outputs/gen_nonuc_ds.nc', 'w')

# MERRA2 RH and T

In [ ]:
##### import MERRA2 datasets for RH and T #####

#import 
T_ds = xr.open_mfdataset('../../MERRA2/2016/GEOS_0.5x0.625_NA/MERRA2/2016/*/MERRA2.2016*.A1.05x0625.NA.nc4', combine = 'by_coords')
RH_ds = xr.open_mfdataset('../../MERRA2/2016/GEOS_0.5x0.625_NA/MERRA2/2016/*/MERRA2.2016*.A3dyn.05x0625.NA.nc4', combine = 'by_coords')

#reduce datasets just to T and RH
T_ds = T_ds['TS']
RH_ds = RH_ds['RH']

In [ ]:
##### Select T and RH in the bounds of our US lat and lon #####
T = {}
RH = {}
for region in utils.lat_lon_dict.keys():
    T[region] = {}
    T[region] = T_ds.sel(
        lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
        lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])
    ).groupby('time.season').mean()
    RH[region] = {}
    RH[region] = RH_ds.sel(
        lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
        lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])
    ).groupby('time.season').mean()

In [ ]:
##### Calculate seasonal mean RH and T by region #####
T_seasonal_mean = {}
RH_seasonal_mean = {}
for region in utils.lat_lon_dict.keys():
    T_seasonal_mean[region] = {}
    RH_seasonal_mean[region] = {}
    for seasons in ['DJF','JJA','MAM','SON']:
        T_seasonal_mean[region][seasons] = T[region].sel(season = seasons).mean().values
        RH_seasonal_mean[region][seasons] = RH[region].sel(season = seasons).mean().values

In [ ]:
##### convert to dataframes ##### 

T_df = pd.DataFrame.from_dict({(i,j): T_seasonal_mean[i][j]
                            for i in T_seasonal_mean.keys() 
                            for j in T_seasonal_mean[i].keys()},
                            orient='index', columns = ['T'])
T_df.index = pd.MultiIndex.from_tuples(T_df.index)

RH_df = pd.DataFrame.from_dict({(i,j): RH_seasonal_mean[i][j]
                            for i in RH_seasonal_mean.keys() 
                            for j in RH_seasonal_mean[i].keys()},
                            orient='index', columns = ['RH'])
RH_df.index = pd.MultiIndex.from_tuples(RH_df.index)



In [ ]:
##### merge dataframes #####
RH_T_df = pd.merge(
    T_df, RH_df, 
    left_index=True, right_index=True)

In [ ]:
##### save dataframe #####
pd.DataFrame.to_csv(RH_T_df, './data/RH_T.csv')

# ISORROPIA Data

In [2]:
#import HEMCO emissions files and choose the first level, combine into one dataset
ds_nonuc = xr.open_zarr('./data/pm_nonuc_ds.zarr')
ds_normal = xr.open_zarr('../model_validation/data/pm_ds.zarr')
ds_normal = ds_normal.sel(model_name = 'MODEL').drop('model_name')
ds = xr.concat([ds_nonuc, ds_normal], pd.Index(['no_nuclear', 'normal'], name='model_name'))

#import RH and T dataframe
RH_T_df = pd.read_csv('./data/RH_T.csv', index_col=[0,1])

#subset our ds into just the species we need
species_list = ['SO4','NH3','HNO3','NIT','NH4']
isorropia_ds = ds[species_list]

ValueError: group not found at path ''

SO4, NH3, NO3, Cl, Ca, K, Mg

In [4]:
#create a dataframe for our isorropia monthly mean values for no nuclear and normal cases, indexed by species, location, season

isorropia_dict = {}
for species in species_list:
    isorropia_dict[species]= {}
    for region in utils.lat_lon_dict.keys():
        isorropia_dict[species][region] = {}
        for season in ['DJF','MAM','JJA','SON']:
            data = isorropia_ds.sel(lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
                                    lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])).groupby('time.season').mean(dim = ['lat','lon','time']).sel(season = season)[species]
            isorropia_dict[species][region][season] = data.values
            
isorropia_df = pd.DataFrame.from_dict({(i,j,k): isorropia_dict[i][j][k] 
                            for i in isorropia_dict.keys() 
                            for j in isorropia_dict[i].keys()
                            for k in isorropia_dict[i][j].keys()},
                            orient='index', columns = ['no_nuclear','normal'])

isorropia_df.index = pd.MultiIndex.from_tuples(isorropia_df.index, names = ['Species','Location', 'Season'])

In [5]:
#convert from mol/mol to mol/m3 by multiplying by P/T*R (mol/m3)
stp_p = 101325 #Pa = kg/m/s^2
R = 8.314 #J/K/mol
isorropia_df['no_nuclear_2x'] = '0'
for species in species_list:
    for region in utils.lat_lon_dict.keys():
            for season in ['DJF','MAM','JJA','SON']:
                isorropia_df.loc[(species,region,season)]['no_nuclear'] *= stp_p/R/RH_T_df.loc[(region,season)]['T']
                isorropia_df.loc[(species,region,season)]['normal'] *= stp_p/R/RH_T_df.loc[(region,season)]['T']


/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
#calculate the total Nitrate and ammonium by molarity
isorropia_df_TNO3 = isorropia_df.loc['HNO3'] + isorropia_df.loc['NIT'] 
isorropia_df_TNH3 = isorropia_df.loc['NH3'] + isorropia_df.loc['NH4'] 

## Run through ISORROPIA
### Only run once to initiate a new ISORROPIA output, otherwise, the output is already made

os.chdir('../../ISORROPIAIIStandalone')
for model in ['no_nuclear', 'normal']:
    for region in ['SE_lat_lon', 'NW_lat_lon', 'NE_lat_lon', 'MW_lat_lon', 'SW_lat_lon']:
            for season in ['DJF','JJA']:
                T_tmp = RH_T_df.loc[(region, season)]['T']
                RH_tmp = RH_T_df.loc[(region, season)]['RH']
                SO4_tmp = isorropia_df.loc['SO4',region,season][model]
                
                os.system(f'mkdir -p {region}_{season}_{model}/')
                os.system(f'cp src/* {region}_{season}_{model}/')
                
                os.chdir(f'{region}_{season}_{model}/')
                print(os.listdir())
                
                cmdprefix = 'cat ISORange.dat | sed -i '
                cmdT = f"-e 's/T(K):       0.0/T(K):       {T_tmp}/' "
                cmdRH = f"-e 's/RH(-):      0.0/RH(-):      {RH_tmp}/' "
                cmdNH3 = f"-e 's/TSO4:       0.0e-9/TSO4:       {SO4_tmp}/' "
                cmdoutput = f"-e 's/Output:     NW_DJF_nonuc_ISOOutput.nc/Output:     ISOOutput_{region}_{season}_{model}_constant_SO4.nc/' "
                cmdsuffix = 'ISORange.dat'
                
                cmd = cmdprefix+cmdT+cmdRH+cmdNH3+cmdoutput+cmdsuffix
                
                os.system(cmd)
                
                os.chdir('../')

os.chdir('../grid_model/ego_nonuclear_project/')

In [ ]:
#### save our data out ####
isorropia_df.to_csv('./data/ISORROPIA_data.csv')